In [294]:
import os
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from nltk.tokenize import RegexpTokenizer
import re

In [295]:
#pointing the source and listing the file names under that path
path = os.chdir('C://Users//ar393556//Documents//Ticket grams//CIdata')
files = os.listdir(path)
files

['chamados_holmes_636815331582475939.xlsx',
 'ChannelOps_May_19_636955917611479013.xlsx',
 'CI 99 Cents v1.0_636626695338656480.xlsx',
 'CI 99 Cents v1.1_636627763103872504.xlsx',
 'CI 99 Cents v1.1_636627769191928504.xlsx',
 'CI 99 Cents v1.1_636627770834452504.xlsx',
 'CI 99 Cents v1.2_636627771331156504.xlsx',
 'CI 99 Cents v1.2_636627779522872504.xlsx',
 'CI 99 Cents v1.3_636632003244997995.xlsx',
 'CI 99 Cents v1.4_636632003663233995.xlsx',
 'CI 99 Cents v1.4_636632008637917995.xlsx',
 'CI 99 Cents_636626064943787079.xlsx',
 'CI 99 Cents_636626066528435079.xlsx',
 'CI 99 Cents_636626880421580480.xlsx',
 'CI AAM_636885158335744764.xlsx',
 'CI Airtel_636854880789369822.xlsx',
 'CI Airtel_636854883641629822.xlsx',
 'CI.xlsx',
 'foobar.txtdic',
 'ticket_db.txtdic',
 'tmp.xlsx']

In [296]:
#Fetching only xlsx files
files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls

['chamados_holmes_636815331582475939.xlsx',
 'ChannelOps_May_19_636955917611479013.xlsx',
 'CI 99 Cents v1.0_636626695338656480.xlsx',
 'CI 99 Cents v1.1_636627763103872504.xlsx',
 'CI 99 Cents v1.1_636627769191928504.xlsx',
 'CI 99 Cents v1.1_636627770834452504.xlsx',
 'CI 99 Cents v1.2_636627771331156504.xlsx',
 'CI 99 Cents v1.2_636627779522872504.xlsx',
 'CI 99 Cents v1.3_636632003244997995.xlsx',
 'CI 99 Cents v1.4_636632003663233995.xlsx',
 'CI 99 Cents v1.4_636632008637917995.xlsx',
 'CI 99 Cents_636626064943787079.xlsx',
 'CI 99 Cents_636626066528435079.xlsx',
 'CI 99 Cents_636626880421580480.xlsx',
 'CI AAM_636885158335744764.xlsx',
 'CI Airtel_636854880789369822.xlsx',
 'CI Airtel_636854883641629822.xlsx',
 'CI.xlsx',
 'tmp.xlsx']

In [297]:
df = pd.DataFrame()


In [298]:
# df = pd.DataFrame({'A': [1, 3], 'B': [3, 4]})
# df.to_excel('tmp.xlsx', index=False)

# pd.read_excel('tmp.xlsx', usecols=['B'])

In [299]:
#Reading the files and appending the data in the DataFrame format
for f in files_xls:
    data = pd.read_excel(f,sheet_name='CI-Data')
    df = df.append(data)

In [300]:
#Taking only TicketID and Summary which are essential

ticket_df = pd.DataFrame(df, columns= ['Summary'])

In [301]:
#Printing top 5 records
ticket_df.head()

,Summary
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [302]:
#no. of records and columns
ticket_df.shape

(380549, 1)

In [303]:
#Dropping null/blank records
ticket_df=ticket_df.dropna()

In [304]:
#New table data shape
ticket_df.shape

(336470, 1)

In [305]:
ticket_df.head()

,Summary
0,Translations stuck in retrieval stage
1,SIS Alert HTTP-pro-cds-presentation-web-llb9.h...
2,SIS Alert HTTP-pro-cds-presentation-web-llb9.h...
3,Request for refresh
4,SIS Alert HTTP-pro-cds-presentation-web-llb9.h...


In [221]:
# ticket_df['Summary']=ticket_df['Summary'][0:1000]

In [306]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
ticket_df['Summary'] = ticket_df['Summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [307]:
ticket_df['Summary'].head()

0                   Translations stuck retrieval stage
1    SIS Alert HTTP-pro-cds-presentation-web-llb9.h...
2    SIS Alert HTTP-pro-cds-presentation-web-llb9.h...
3                                      Request refresh
4    SIS Alert HTTP-pro-cds-presentation-web-llb9.h...
Name: Summary, dtype: object

In [308]:
# generate unigrams 
unigrams  = (ticket_df['Summary'].str.lower()
                .str.replace(r'[^a-z\s]', '')
                .str.split(expand=True)
                .stack())

In [309]:
#Generate bigrams by shifting unigrams
bigrams=unigrams+' '+unigrams.shift(-1)

In [310]:
bigrams[0:5]

0  0    translations stuck
   1       stuck retrieval
   2       retrieval stage
   3             stage sis
1  0             sis alert
dtype: object

In [311]:
# generate trigrams by concatenating unigram and bigram columns
trigrams = bigrams + ' ' + unigrams.shift(-2)


In [312]:
trigrams[0:5]

0  0                         translations stuck retrieval
   1                                stuck retrieval stage
   2                                  retrieval stage sis
   3                                      stage sis alert
1  0    sis alert httpprocdspresentationwebllbhoustonh...
dtype: object

In [313]:
# generate trigrams by concatenating unigram and trigram columns
fourgrams = trigrams + ' ' + unigrams.shift(-4)


In [314]:
fourgrams[0:5]

0  0                     translations stuck retrieval sis
   1                          stuck retrieval stage alert
   2    retrieval stage sis httpprocdspresentationwebl...
   3                         stage sis alert wepoprdhhttp
1  0    sis alert httpprocdspresentationwebllbhoustonh...
dtype: object

In [315]:
print(len(bigrams))
print(len(trigrams))

print(len(fourgrams))



1374688
1374688
1374688


In [243]:
unigrams_list = unigrams.values.tolist() 

In [241]:
bigram_list[3]

'files stuck'

In [244]:
from gensim import corpora
#docs= ticket_df['Summary_preprocess'].values.tolist()
unigram_text = [[i for i in doc.split()] for doc in unigrams_list]
# print(texts)
dictionary = corpora.Dictionary(unigram_text)

In [249]:
print(dictionary)

Dictionary(6285 unique tokens: ['ctrlaltdel', 'necessary', 'loaning', 'bording', 'maker']...)


In [251]:
#[dictionary[id] for id in dictionary]

In [255]:
from nltk.tokenize import word_tokenize

In [262]:
import nltk.data
nltk.download('punkt')
example = ['Mary had a little lamb' , 
           'Jack went up the hill' , 
            'Jill followed suit' ,    
            'i woke up suddenly' ,
           'it was a really bad dream...']
tokenized_sents = [word_tokenize(i) for i in example]
for i in tokenized_sents:
    print (i)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ar393556\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
['Mary', 'had', 'a', 'little', 'lamb']
['Jack', 'went', 'up', 'the', 'hill']
['Jill', 'followed', 'suit']
['i', 'woke', 'up', 'suddenly']
['it', 'was', 'a', 'really', 'bad', 'dream', '...']


In [264]:
from gensim import corpora
#docs= ticket_df['Summary_preprocess'].values.tolist()
#unigram_text = [[i for i in doc.split()] for doc in tokenized_sents]
# print(texts)
dictionary = corpora.Dictionary(tokenized_sents)

In [267]:
[dictionary[id] for id in dictionary]

['went',
 'little',
 'really',
 'bad',
 'woke',
 'it',
 'up',
 'Jack',
 'suit',
 'Jill',
 'dream',
 'had',
 'lamb',
 'hill',
 'a',
 'the',
 '...',
 'was',
 'suddenly',
 'followed',
 'Mary',
 'i']

In [268]:
def ngrams(s, n=2, i=0):
     while len(s[i:i+n]) == n:
            yield s[i:i+n]
            i += 1
txt = 'Python is one of the awesomest languages'

unigram = ngrams(txt.split(), n=1)
list(unigram)


bigram = ngrams(txt.split(), n=2)
list(bigram)

trigram = ngrams(txt.split(), n=3)
list(trigram)


[['Python', 'is', 'one'],
 ['is', 'one', 'of'],
 ['one', 'of', 'the'],
 ['of', 'the', 'awesomest'],
 ['the', 'awesomest', 'languages']]

In [275]:
from gensim import corpora
unigram=unigram.values.tolist()
#unigram_text = [[i for i in doc.split()] for doc in tokenized_sents]
# print(texts)
dictionary = corpora.Dictionary(unigram)

AttributeError: 'generator' object has no attribute 'values'

In [276]:
[dictionary[id] for id in dictionary]

[]

In [281]:
from gensim.models import Phrases
documents = ["the mayor of new york was there", "machine learning can be useful sometimes"]

sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream)
#trigram=Phrases(bigram)
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
print(bigram[sent])

['the', 'mayor', 'of', 'new', 'york', 'was', 'there']


C:\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [282]:
# print (bigram.vocab)
[bigram[id] for id in bigram]

C:\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


TypeError: 'int' object is not iterable

In [11]:
ticket_df['words']=ticket_df['Summary'].apply(lambda x:len(str(x).split(" ")))

ticket_df.head()

,TicketID,Summary,words
0,INC162261268,Alert: PRIORITY 2 - Trickle Files Stuck: STORE_67,8
1,INC162261269,Alert: PRIORITY 2 - Back Office Trickle Files ...,10
2,INC162261270,Express Server 161 Disk D: 100%,6
3,INC162261271,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9
4,INC162261272,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9


In [12]:
# ticket_df['Summary'].hist(bins=2)
# plt.show()

In [13]:
ticket_df['Summary_preprocess']= ticket_df['Summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))


In [14]:
ticket_df['Summary_preprocess']= ticket_df['Summary_preprocess'].str.replace('[^\w\s]','')
ticket_df['Summary_preprocess'].head()

0       alert priority 2  trickle files stuck store_67
1    alert priority 2  back office trickle files st...
2                        express server 161 disk d 100
3    alert priority 1  xps service has stopped stor...
4    alert priority 1  xps service has stopped stor...
Name: Summary_preprocess, dtype: object

In [15]:
ticket_df['Summary_preprocess']= ticket_df['Summary_preprocess'].str.replace('[\d+]','')


In [16]:
ticket_df.head()

,TicketID,Summary,words,Summary_preprocess
0,INC162261268,Alert: PRIORITY 2 - Trickle Files Stuck: STORE_67,8,alert priority trickle files stuck store_
1,INC162261269,Alert: PRIORITY 2 - Back Office Trickle Files ...,10,alert priority back office trickle files stu...
2,INC162261270,Express Server 161 Disk D: 100%,6,express server disk d
3,INC162261271,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service has stopped store_
4,INC162261272,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service has stopped store_


In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
ticket_df['Summary_preprocess'] = ticket_df['Summary_preprocess'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# ticket_df['Summary_preprocess'].head()

In [18]:
ticket_df.head()

,TicketID,Summary,words,Summary_preprocess
0,INC162261268,Alert: PRIORITY 2 - Trickle Files Stuck: STORE_67,8,alert priority trickle files stuck store_
1,INC162261269,Alert: PRIORITY 2 - Back Office Trickle Files ...,10,alert priority back office trickle files stuck...
2,INC162261270,Express Server 161 Disk D: 100%,6,express server disk
3,INC162261271,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service stopped store_
4,INC162261272,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service stopped store_


In [19]:
freq = pd.Series(' '.join(ticket_df['Summary_preprocess'] ).split()).value_counts()[-250:]
freq

hsr                              1
downlatency                      1
roach                            1
cerdian                          1
images                           1
mailtockchowkingnet              1
mahendaran                       1
merch                            1
videop                           1
ceiling                          1
ramirezs                         1
nitem                            1
qlacposrule                      1
laeprtbp                         1
comes                            1
minecraftcorezombiewaccesasst    1
bcp                              1
clien                            1
resetams                         1
connectunable                    1
amplifier                        1
excess                           1
storemanager                     1
pxs                              1
followup                         1
either                           1
beifits                          1
pn                               1
modified            

In [ ]:
# tf1 = (ticket_df['Summary_preprocess']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
# tf1.columns = ['words','tf']


In [67]:
docs= ticket_df['Summary_preprocess'].values.tolist()

In [68]:
docs

['alert priority trickle files stuck store_',
 'alert priority back office trickle files stuck store_',
 'express server disk',
 'alert priority xps service stopped store_',
 'alert priority xps service stopped store_',
 'alert priority xps service stopped store_',
 'alert priority download files stuck store_',
 'alert priority dbs service running register',
 'back office pc slow',
 'register closed',
 'alert priority status update received store_',
 'alert priority status update received store_',
 'sm taking credentials',
 'r frozen',
 'warning disk space utilization production server ladsappp',
 'sm taking password',
 'alert priority status update received store_',
 'ceridian question',
 'bo printer working properly',
 'alert priority trickle files stuck store_',
 'alert priority trickle files stuck store_',
 'boc vigile application error',
 'sales tax rate change effective sunday',
 'pos password reset',
 'r frozen',
 'ceridian tc error',
 'alert priority trickle files stuck store_'

In [70]:
new_words=[]
for sent in docs:
    temp=sent.split()
    for words in temp:
        
        if len(words)<=2:
            continue
        else:
            new_words.append(words)

In [71]:
new_words

['alert',
 'priority',
 'trickle',
 'files',
 'stuck',
 'store_',
 'alert',
 'priority',
 'back',
 'office',
 'trickle',
 'files',
 'stuck',
 'store_',
 'express',
 'server',
 'disk',
 'alert',
 'priority',
 'xps',
 'service',
 'stopped',
 'store_',
 'alert',
 'priority',
 'xps',
 'service',
 'stopped',
 'store_',
 'alert',
 'priority',
 'xps',
 'service',
 'stopped',
 'store_',
 'alert',
 'priority',
 'download',
 'files',
 'stuck',
 'store_',
 'alert',
 'priority',
 'dbs',
 'service',
 'running',
 'register',
 'back',
 'office',
 'slow',
 'register',
 'closed',
 'alert',
 'priority',
 'status',
 'update',
 'received',
 'store_',
 'alert',
 'priority',
 'status',
 'update',
 'received',
 'store_',
 'taking',
 'credentials',
 'frozen',
 'warning',
 'disk',
 'space',
 'utilization',
 'production',
 'server',
 'ladsappp',
 'taking',
 'password',
 'alert',
 'priority',
 'status',
 'update',
 'received',
 'store_',
 'ceridian',
 'question',
 'printer',
 'working',
 'properly',
 'alert',
 '

In [72]:
from collections import Counter

# helloString = ['hello', 'world', 'world', 'how', 'are', 'you', 'doing', 'today']
counter = Counter(new_words)

In [76]:
refined_words=[]
for letter, count in counter.most_common(3000): 
    print('%s: %d' % (letter, count)) 

password: 10850
reset: 10237
dayforce: 8573
alert: 5995
store_: 5528
priority: 5475
reg: 5164
register: 4370
error: 3611
request: 3540
printer: 3471
working: 3254
store: 3248
pos: 3138
unable: 2528
frozen: 2470
item: 2330
account: 2255
scanning: 1964
toner: 1911
unlock: 1881
stuck: 1874
status: 1786
issue: 1747
files: 1662
server: 1631
update: 1548
received: 1547
responding: 1431
access: 1421
informational: 1414
pinpad: 1408
cannot: 1361
email: 1311
service: 1138
workstation: 1130
ping: 1113
issues: 1081
user: 1064
trickle: 1062
employee: 1039
payroll: 1033
pin: 1031
office: 1003
pad: 992
running: 987
sam: 976
slow: 931
drawer: 918
critical: 907
call: 900
back: 880
ism: 869
process: 865
open: 857
locked: 848
accuterm: 830
manager: 826
casrg: 790
offline: 773
active: 753
link: 738
time: 735
new: 728
period: 726
registers: 710
eod: 708
posdm: 702
primary: 659
stopped: 658
report: 638
order: 627
sales: 620
monitor: 619
directory: 617
computer: 610
opening: 597
information: 592
check: 588


In [23]:
counter

Counter({'phone working': 6,
         '': 6,
         'reg printer woprking': 1,
         'phone company needs contacted': 1,
         'er daily jedi report': 9,
         'rester frozen': 1,
         'back office pc task bar correct place': 1,
         'hh issue responding asset': 1,
         'provide transport new import codes': 2,
         'pos scale reg': 1,
         'er posdm alert posdm sales reported store': 4,
         'reg drawer issues': 5,
         'information rh complaints': 1,
         'high jump error ocurred': 1,
         'speaker micro phone office store': 1,
         'customer server request': 1,
         'scannergun locked': 1,
         'bo frozen internet slowness issue': 1,
         'time clock unable enroll user': 1,
         'inventory question': 1,
         'scale issue register': 1,
         'experiencing power outage': 1,
         'back office internet access': 1,
         'register acting slowscanning items': 1,
         'server offline rg': 1,
         'reg o

In [ ]:
for i in t    
    
    return (lst[0].split()) 
  
# Driver code 
lst =  ["Geeks For geeks"] 
print( convert(lst)) 

In [20]:
# freq = list(freq.index)
# ticket_df['Summary_preprocess'] = ticket_df['Summary_preprocess'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))


In [21]:
# ticket_df.head()

,TicketID,Summary,words,Summary_preprocess
0,INC162261268,Alert: PRIORITY 2 - Trickle Files Stuck: STORE_67,8,alert priority trickle files stuck store_
1,INC162261269,Alert: PRIORITY 2 - Back Office Trickle Files ...,10,alert priority back office trickle files stuck...
2,INC162261270,Express Server 161 Disk D: 100%,6,express server disk
3,INC162261271,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service stopped store_
4,INC162261272,Alert: PRIORITY 1 - XPS Service has Stopped: S...,9,alert priority xps service stopped store_


C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [46]:
from gensim import corpora
docs= ticket_df['Summary_preprocess'].values.tolist()
texts = [[i for i in doc.lower().split()] for doc in docs]
print(texts)
dictionary = corpora.Dictionary(texts)

In [47]:
dictionary.save('ticket_db.txtdic')

[['alert', 'priority', 'trickle', 'files', 'stuck', 'store_'], ['alert', 'priority', 'back', 'office', 'trickle', 'files', 'stuck', 'store_'], ['express', 'server', 'disk'], ['alert', 'priority', 'xps', 'service', 'stopped', 'store_'], ['alert', 'priority', 'xps', 'service', 'stopped', 'store_'], ['alert', 'priority', 'xps', 'service', 'stopped', 'store_'], ['alert', 'priority', 'download', 'files', 'stuck', 'store_'], ['alert', 'priority', 'dbs', 'service', 'running', 'register'], ['back', 'office', 'pc', 'slow'], ['register', 'closed'], ['alert', 'priority', 'status', 'update', 'received', 'store_'], ['alert', 'priority', 'status', 'update', 'received', 'store_'], ['sm', 'taking', 'credentials'], ['r', 'frozen'], ['warning', 'disk', 'space', 'utilization', 'production', 'server', 'ladsappp'], ['sm', 'taking', 'password'], ['alert', 'priority', 'status', 'update', 'received', 'store_'], ['ceridian', 'question'], ['bo', 'printer', 'working', 'properly'], ['alert', 'priority', 'trickle'

In [51]:
len(dictionary)

6236

In [42]:
# from textblob import TextBlob
# sample=ticket_df['Summary_preprocess'][:25].apply(lambda x: str(TextBlob(x).correct()))

In [43]:
# sample_comp=ticket_df['Summary_preprocess'][:25]

In [78]:
from gensim.models import Phrases

from gensim.models.phrases import Phraser

In [85]:
documents =["the mayor of new york was there machine learning can be useful sometimes new york mayor was present"]

sentence_stream = [doc.split(" ") for doc in documents]
print(sentence_stream)

bigram = Phrases(sentence_stream, min_count=1, threshold=2, delimiter=b' ')

bigram_phraser = Phraser(bigram)





[['the', 'mayor', 'of', 'new', 'york', 'was', 'there', 'machine', 'learning', 'can', 'be', 'useful', 'sometimes', 'new', 'york', 'mayor', 'was', 'present']]


In [89]:
print(bigram_phraser)

In [90]:
for sent in sentence_stream:
    tokens_ = bigram_phraser[sent]

    print(tokens_)

['the', 'mayor', 'of', 'new york', 'was', 'there', 'machine', 'learning', 'can', 'be', 'useful', 'sometimes', 'new york', 'mayor', 'was', 'present']


In [93]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import models



docs = ['new york is is united states', 'new york is most populated city in the world','i love to stay in new york']

token_ = [doc.split(" ") for doc in docs]
bigram = Phrases(token_, min_count=1, threshold=2,delimiter=b' ')


bigram_phraser = Phraser(bigram)

bigram_token = []
for sent in token_:
    bigram_token.append(bigram_phraser[sent])

In [96]:
bigram_token[0]


['new york', 'is', 'is', 'united', 'states']

In [319]:
from nltk import ngrams

sentence = 'this is a foo bar sentences and i want to ngramize it'

n = 2
sixgrams = ngrams(sentence.split(), n)

for grams in sixgrams:
    print(grams)

('this', 'is')
('is', 'a')
('a', 'foo')
('foo', 'bar')
('bar', 'sentences')
('sentences', 'and')
('and', 'i')
('i', 'want')
('want', 'to')
('to', 'ngramize')
('ngramize', 'it')
